In [326]:
# # ================ Demand System ================
# mutable struct DemandSystem
#     Nodes # 2N length vector of xy-coords per node
#     F     # N length vector of demand per node

#     c # Initial cost to build a station
#     f # Scaling cost to scale a station

#     function DemandSystem(Nodes_in, F_in, c_in, f_in)
#         new(Nodes_in, F_in, c_in, f_in)
#     end
# end;

# # --- Gets the coordinates of the ith node ---
# function nCrd(M::DemandSystem, i)
#     return M.Nodes[2*i-1], M.Nodes[2*i]
# end;

# # --- Gets the demand of the ith node ---
# function nDmd(M::DemandSystem, i)
#     return M.F[i]
# end;


# # ================ Design Variable ================
# # --- Gets the coordinates of the kth station ---
# function sCrd(x, k)
#     return x[3*k-2], x[3*k-1]
# end;

# # --- Gets the capacity of the kth station ---
# function sCap(x, k)
#     return x[3*k]
# end;

# # --- Gets the number of stations built ---
# # function sB(x)
# #     return x[end]
# # end;

# # --- Quantizes the b elements to be in 0,...,M ---
# function correct(x)
#     M, b = (length(x)-1)/3, round(x[end])

#     b = b < 1.0 ? 1.0 :
#         b > M ? float(M) :
#         b

#     x[end] = b
#     return x
# end

# # --- Corrects an array of design variables ---
# # function correctArray(X)
# #     X_out = X
# #     len = length(X)
# #     for j = 1:len
# #         X_out[j] = correct(X[j])
# #     end
# #     return X_out
# # end

# # --- Takes a matrix of uncorrected design variables and ---
# #     returns a list of corrected variables 
# function correctMatrix(M)
#     x_out = []
#     row, col = size(M)

#     for j = 1:col
#         x = correct(M[:,j])
#         push!(x_out , x)
#     end

#     return x_out
# end


# # ================ Objective Function ================
# # --- OBJECTIVE FUNCTION: Total cost of all built stations ---
# function costFunc(M::DemandSystem, x)
#     b, sum = Int(x[end]), 0

#     for k = 1:b
#         sum += c + f*sCap(x, k)
#     end
#     return sum
# end;


# # ================ Constraint Functions ================
# # --- Squared distance between a node and a station ---
# function i_kDist(nu, nv, sx, sy)
#     return (nu-sx)*(nu-sx) + (nv-sy)*(nv-sy)
# end;

# # --- INEQUALITY CONSTRAINT i: The ith node's demand is met by the stations ---
# #       (  There are N constraints of this form. One for each i = 1,...,N  )
# #       (  In the form of g_i(x) <= 0  )
# function nodeConstraint_i(M::DemandSystem, x, i)
#     nu, nv = nCrd(M, i)
#     b, sum = Int(x[end]), 0
    
#     for k = 1:b
#         sx, sy = sCrd(x, k)
#         sz     = sCap(x, k)

#         dist = i_kDist(nu, nv, sx, sy)
#         sum += sz/dist
#     end
    
#     return nDmd(M, i) - sum
# end;

# # --- EQUALITY CONSTRAINT: The number of stations built is an integer in 1,...,M ---
# #       (  In the form of h(x) = 0  )
# # function stationConstraint_k(x)
# #     M = (length(x)-1)/3
# #     b = x[end]
# #     for k = 1:M
# #         b *= b-k
# #     end
# #     return (1/M)*b
# # end;


# # ================ Reading Data ================
# # --- Takes a design variable and interprates it into meaningful data ---
# #       (  Returns list of coordinates and capacity for built stations only  )
# #       (  Returns [ (x,y,z), ..., (x,y,z) ] for built stations only  )
# #       (  Must be in valid for, i.e. b = 0,...,M  )
# function readVariable(x)
#     result = []
#     b = Int(x[end])
#     for k = 1:b
#         sx, sy = sCrd(x, k)
#         sz = sCap(x, k)

#         push!(result, (sx, sy, sz))
#     end

#     return result
# end;

In [327]:
# # ================ Cross Entropy Method ================
# using Distributions
# function getDist(x_elite)
#     m = length(x_elite)
#     l = length(x_elite[1])
#     mean = zeros(l)
#     for i = 1:m
#         mean += x_elite[i]
#     end
#     mean *= 1.0/float(m)

#     cov = zeros(l,l)
#     for i = 1:m
#         v = x_elite[i] - mean
#         cov += v*v'
#     end
#     cov *= 1.0/float(m)

#     # return MultivariateNormal(mean, cov)
#     return MvNormal(mean, cov)
# end


# function crossEntropy(func, P, k_max, m=100, m_elite=10)
#     for k = 1:k_max
#         x = correctMatrix(rand(P, m))
#         ord = sortperm( [func(x[i]) for i in 1:m] )
#         # P = getDist(x[ord[1:m_elite]])

#         # return P
#         return x[ord[1:m_elite]]
#     end
#     # return P
# end


# # ================ Cost Function with barrier ================
# function costWithInverseBarrier(M::DemandSystem, x, p)
#     N = Int(length(M.Nodes)/2)
#     barrier = 0

#     for i = 1:N
#         barrier -= 1/nodeConstraint_i(M,x,i)
#     end
#     barrier *= 1/p

#     return costFunc(M,x) + barrier
# end;

In [328]:
include("/Users/benjaminrutkowski/School Code Projects/CS268-Final-Project/MODEL_TWO/objective_function.jl")
include("/Users/benjaminrutkowski/School Code Projects/CS268-Final-Project/Cross_Entropy/cross_entropy.jl")

inversePenalty (generic function with 1 method)

In [329]:
# ================ TEST MODEL ================
using Distributions
using LinearAlgebra
Nodes = [
    1.0, 1.0, 
    1.0, 2.0, 
    2.0, 1.0, 
    2.0, 2.0];

F     = [4, 4, 4, 4];
c, f = 10, 5

sys = DemandSystem(Nodes, F, c, f)

x = [
    1.3, 1.4, 0.5,
    1.7, 1.9, 0.4,
    2.0
]
x = correct(x)

mean1 = [1.0, 2.0, 3.0, 4.0, 1.0, 2.0, 1.4]
cov1 = 4.0*I(7)
dist = MultivariateNormal(mean1, cov1);

In [330]:
using Distributions
# println(quadPenalty(sys, x), "\n")
# println(inversePenalty(sys, x))

mean1 = [1.0, 3.0, 5.0]
mean2 = [3.2, 4.1, 4.0]
cov1 = 100*I(3)
cov2 = 100*I(3)
norms = [MvNormal(mean1, cov1), MvNormal(mean2, cov2)]
bi = Normal(2, 1)
split = SplitDist(norms, bi)
samp = sample(split, 10)

for i = 1:10
    # myzfun = z -> costFunc(sys, zToX()) + quadPenalty(sys, x)
end

# TODO: Add a new 

# myfunc = x -> costFunc(sys, x) + quadPenalty(sys, x)
# P = subCrossEntropy(myfunc, split, 100, 100, 10)
# s = sample(P,10)

#361 (generic function with 1 method)